# Compound

> Fill in a module description here

In [ ]:
# | default_exp compound.core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# | export
from dataclasses import dataclass
import chemlib

from exex.imports import *
from exex.core.all import *
from exex.utils import *
from exex.compound.law import *

### Compound

In [ ]:
# | export
@docs
class Compound(Matter):

    LAWS = [MassMoleRatio]

    def __init__(self, formula: str) -> None:  # the chemical formula
        super().__init__()

        compound = chemlib.Compound(formula)
        self.add_laws = [MassMoleRatio]
    
        self.set_elements(compound.elements)

        self.formula = compound.formula
        self.str_formula = formula
        
        self.coefficient = compound.coefficient
        self.occurences = compound.occurences

        self._setup_laws([MassMoleRatio])
    
    @property
    def formula(self) -> str:
        return self._unicode_formula
    
    @formula.setter
    def formula(self, formula: str) -> None:
        self._unicode_formula = formula
    
    @property
    def snake_name(self) -> str:  # return the snake name style
        return self.str_formula
    
    @property

    def get_data(self, time: int, name: str):  # the time  # the property name
        if not name in self.properties:
            return "The property don't exist"
        pass

    def __repr__(self):
        return f"Compound({self.formula})"

    _docs = dict(cls_doc="Compound",
                 get_data="",
                 snake_name="the snake name style"
                )

In [ ]:
#| export
@patch(as_prop=True)
def elements(self: Matter): # return elements of the compound
    return self._elements

In [ ]:
#| export
@patch
def set_elements(self: Matter, elements):
    self._elements = elements

In [ ]:
# @patch(as_prop=True)
# def formula(self: Matter):
#     return self._unicode_formula

In [ ]:
#| export
@patch(as_prop=True)
def coeffs(self: Compound):
    return self._coeffs

In [ ]:
#| export
@patch
def info(self: Compound, **kwargs): # the info
    dta = {}

    for k, v in self.properties.items():
        # data_point = {}
        # print(v._data)
        key = k
        # if v.unit:
        #     key += f' ({v.unit})'

        dta[key] = v._data

    df = pd.DataFrame(data=dta, **kwargs)
    df.index.name = "Time"
    return df.sort_index()

In [ ]:
class A:
    COMPOUNDS = [10, 20, 30]

    def __init__(self):
        self.compounds = [1, 2, 3]

In [ ]:
class B(A):
    COMPOUNDS = [40, 50, 60]

    def __init__(self):
        super().__init__()
        self.compounds = [4, 5, 6]

    @classmethod
    def base(cls):
        return cls.__bases__

In [ ]:
b = B()

In [ ]:
b.base()

(__main__.A,)

In [ ]:
A.__bases__

(object,)

In [ ]:
B.__bases__

(__main__.A,)

In [ ]:
b.__dict__

{'compounds': [4, 5, 6]}

In [ ]:
H2O = Compound("H2O")

In [ ]:
H2O

Compound(H₂O₁)

In [ ]:
H2O.formula

'H₂O₁'

In [ ]:
H2O.get_law("mass_mole_ratio", t=2)

Eq(M_H2O-2*n_H2O-2, m_H2O-2)

In [ ]:
H2O.snake_name

'H2O'

In [ ]:
# | hide
test_eq(H2O.occurences["H"], 2)
test_eq(len(H2O.elements), 3)
test_eq(H2O.snake_name, "H2O")

In [ ]:
# | hide
test_eq(H2O.properties["mass"].unit, Unit.MASS)
test_eq(H2O.properties["mole"].unit, Unit.MOLE)